In [ ]:
import copy
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

def last_modified(opts, remote_path):
    info = get_info_remote(get_wdclient(opts), remote_path.as_posix())
    format_ = '%a, %d %b %Y %H:%M:%S GMT'
    return datetime.datetime.strptime(info['modified'], format_)

# Macro-Pipeline AHN2 Workflow - Retiling (Object Points)

## Set Run-Specific Input

Fill in the username/password for the SURF dCache. Choose whether you want to i) run all input files, ii) run the only input files listed in `filename`, or iii) run the input that was updated since the last workflow run.

In [ ]:
remote_path_root = pathlib.Path('/pnfs/grid.sara.nl/data/projects.nl/eecolidar')

# dCache path to raw LAZ files 
remote_path_input = remote_path_root / '00_Data/ALS/Netherlands/ahn2/objects'

# dCache path where to copy retiled LAZ files
remote_path_output = remote_path_root / '01_Escience/ALS/Netherlands/ahn2/retiled'

run = 'all' # 'all', 'updated', 'from_file'
filename = 'retile_failed.json'  # if run is 'from_file', set name of file with input file names
assert run in ['all', 'updated', 'from_file']

In [ ]:
webdav_login = input('WebDAV username: ')
webdav_password = getpass.getpass('WebDAV password: ')
if run == 'updated':
    last_run = datetime.datetime.strptime(input('Date last run (YYYY-MM-DD): '), '%Y-%m-%d')

## Check Connection to Remote Storage

In [ ]:
wd_opts = {
    'webdav_hostname': 'https://webdav.grid.surfsara.nl:2880',
    'webdav_login': webdav_login,
    'webdav_password': webdav_password
}
assert get_wdclient(wd_opts).check(remote_path_root.as_posix())

In [ ]:
laz_files = [f for f in list_remote(get_wdclient(wd_opts), remote_path_input.as_posix())
             if f.lower().endswith('.laz')]
print('Found: {} LAZ files'.format(len(laz_files)))
if run == 'updated':
    laz_files = [f for f in laz_files if last_modified(wd_opts, remote_path_input/f) > last_run]
elif run == 'from_file':
    with open(filename, 'r') as f:
        laz_files_read = json.load(f)
    # check whether all files are available on dCache
    assert all([f in laz_files for f in laz_files_read]), f'Some of the files in {filename} are not in remote dir'
    laz_files = laz_files_read
print('Retrieve and retile: {} LAZ files'.format(len(laz_files)))

## Setup Cluster

Setup Dask cluster used for the macro-pipeline calculation.

In [ ]:
local_tmp = pathlib.Path('/data/local/tmp')

nprocs_per_node = 2 

# start the cluster
scheduler_node = 'node1'
hosts = [f'node{i}' for i in range(1, 11)]
cluster = SSHCluster(hosts=[scheduler_node] + hosts, 
                     connect_options={'known_hosts': None, 
                                      'username': 'ubuntu', 
                                      'client_keys': '/home/ubuntu/.ssh/id_rsa'},
                     worker_options={'nthreads': 1, 
                                     'nprocs': nprocs_per_node,
                                     'memory_limit': 0,
                                     'local_directory': local_tmp/'dask-worker-space'}, 
                     scheduler_options={'dashboard_address': '8787'})
cluster

## Retiling

The raw point-cloud files are downloaded and retiled to a regular grid.

In [ ]:
# details of the retiling schema
grid = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}

In [ ]:
# setup input dictionary to configure the retiling pipeline
retiling_input = {
    'setup_local_fs': {'tmp_folder': local_tmp.as_posix()},
    'pullremote': remote_path_input.as_posix(),
    'set_grid': grid,
    'split_and_redistribute': {},
    'validate': {},
    'pushremote': remote_path_output.as_posix(),
    'cleanlocalfs': {}
}

# write input dictionary to JSON file
with open('retile.json', 'w') as f:
    json.dump(retiling_input, f)

In [ ]:
macro = MacroPipeline()

# add pipeline list to macro-pipeline object and set the corresponding labels
macro.tasks = [Retiler(file).config(retiling_input).setup_webdav_client(wd_opts) for file in laz_files]
macro.set_labels([os.path.splitext(file)[0] for file in laz_files])

macro.setup_cluster(cluster=cluster)

# run! 
macro.run()

# save outcome results and check that no error occurred before continuing
macro.print_outcome(to_file='retile.out')

failed = macro.get_failed_pipelines()
if failed:
    with open('retile_failed.json', 'w') as f:
        json.dump([pip.label + '.laz' for pip in failed], f)
    raise RuntimeError('Some of the pipelines have failed')

In [ ]:
print('hello')

## Terminate cluster

In [ ]:
macro.shutdown()

## Troubleshooting 

### Cancel all jobs and restart the notebook

Copy and paste these lines in a separate Python shell. If the Dask dashboard shows that some tasks are still queued to be processed, run the lines again - this should clear the scheduler up and give back control to the current notebook. Normally proceed to terminate the cluster and restart the notebook.

In [ ]:
from dask.distributed import Client, Future
client = Client('tcp://145.100.59.123:8786')
futures = [Future(key) for key in client.who_has().keys()]
client.cancel(futures)